In [ ]:
import glob
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import os

import scanpy as sc
import seaborn as sns

from scroutines import basicu

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
from tqdm import tqdm

In [ ]:
f1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome.h5ad'
f2 = '/u/home/f/f7xiesnm/v1_multiome/multiome_cell_sex_assignment_saumya.csv'
meta  = sc.read(f1, backed='r').obs
df_sex = pd.read_csv(f2)
meta = meta.join(df_sex.set_index('cell'))
meta

In [ ]:
meta['Age'] = meta['Age'].astype(str)
meta['Sample'] = meta['Sample'].astype(str)
meta['Subclass'] = meta['Subclass'].astype(str)
print(meta.shape)

# filter sex assignment (remove undetermined)
meta = meta[meta['sex']!='nan']
print(meta.shape)

# filter condition - at least 2 samples having both sex
meta = meta[~meta['Age'].isin(['P14', 'P21'])]
print(meta.shape)

# filter subclass - at least 10 cells in any sample
subclass_abundance = meta.groupby(['Subclass', 'Sample']).size().unstack().fillna(0)
subclass_abundance_pass = subclass_abundance[subclass_abundance.min(axis=1) > 10]
subclasses = subclass_abundance_pass.index.values
meta = meta[meta['Subclass'].isin(subclasses)]
print(meta.shape)

In [ ]:
uniq_subclasses = np.unique(meta['Subclass'])
uniq_conditions = np.unique(meta['Age'])
print(uniq_subclasses)
print(uniq_conditions)

In [ ]:
meta_counts = meta.groupby(['Sample', 'sex', 'Subclass']).size().unstack().fillna(0)
meta_counts

In [ ]:
meta_counts.describe()

In [ ]:
meta_counts2 = (meta.groupby(['Subclass', 'Age', 'Sample', 'sex']).size()
                    .groupby(['Subclass', 'Age', 'Sample']).min() # smaller among M & F
                    .groupby(['Subclass', 'Age']).max() # largest sample
                )
case_thresholds = meta_counts2 > 100
case_thresholds_map = case_thresholds.unstack().T

sns.heatmap(case_thresholds_map, cmap='rocket_r')

In [ ]:
meta_counts2.unstack().T

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/annot/gencode.vM25.genebody.bed'
df_annot = pd.read_csv(f, sep='\t', header=None)
df_annot

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_sexual_dimorphism/'
!ls $ddir

In [ ]:
df_merge = []
for (subclass, condition), isok in case_thresholds.items():
    
    subclass_cure = subclass.replace('/', '')
    
    filename = os.path.join(ddir, f'{subclass_cure}_{condition}.csv')
    
    df = pd.read_csv(filename, index_col=0)
    df['max_cp10k'] = np.max(df.iloc[:,5:], axis=1)
    
    x = df['log2fc']
    y = df['qval']
    z = df['max_cp10k']

    x_th = np.log2(2)
    y_th = 0.05
    z_th = 0.1

    cond_sig = np.all([np.abs(x) > x_th, 
                               y < y_th,
                               z > z_th,
                       df['converged'].values,
                      ], axis=0)

    
    print(subclass, condition, np.sum(cond_sig))
    df_sig = df[cond_sig][['gene', 'log2fc', 'qval', 'converged', 'max_cp10k']].copy()
    df_sig['condition'] = condition
    df_sig['subclass'] = subclass
    
    if not isok: 
        print('skip:', subclass, condition)
    else:
        df_merge.append(df_sig)
    
df_merge = pd.concat(df_merge)
df_merge

In [ ]:
# subclasses = ['L2/3', 'L4', 'L5IT', 'L5NP', 'L5PT', 'L6CT', 'L6IT', 'L6b', 'Lamp5', 'Pvalb', 'Sst', 'Vip', 'Astro', 'Micro', 'OD', 'OPC', ]
subclasses = df_merge['subclass'].unique()

In [ ]:
df_merge_add_all = []

for subclass in subclasses: # ['L23', 'L4', 'L5IT', 'L6IT']: #subclasses:
    df_merge_add = df_merge[df_merge['subclass']==subclass].groupby('gene')['condition'].agg(lambda x: ' '.join(x)).sort_values()
    df_merge_add = df_merge_add.to_frame().join(df_annot.set_index(3)[0])#.sort_values('condition')
    df_merge_add['n_condition'] = df_merge_add['condition'].apply(lambda x: len(x.split(' ')))
    # df_merge_add['n_condition_nr'] = df_merge_add['condition'].apply(lambda x: len([_x for _x in x.split(' ') if not _x.endswith('DR')]))
    df_merge_add = df_merge_add.sort_values('n_condition', ascending=False)
    
    df_merge_add = df_merge_add[df_merge_add['n_condition'] > 1]
    df_merge_add['subclass'] = subclass
    
    df_merge_add_all.append(df_merge_add)

df_merge_add_all = pd.concat(df_merge_add_all)
df_merge_add_all

In [ ]:
df_merge_add_all['subclass'].value_counts()

In [ ]:
df_merge_add_all.to_csv('check.csv')